# Stage 04: Data Acquisition and Ingestion Homework

This notebook demonstrates data acquisition from APIs and web scraping, with proper validation and file naming conventions.

## Setup and Imports

In [1]:
import os, json, datetime as dt
from pathlib import Path
from typing import Dict
import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

print("All imports successful!")

All imports successful!


## Project Setup

In [2]:
DATA_RAW = Path("data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

print(f"Data directory created: {DATA_RAW}")
print(f"Alpha Vantage API key available: {bool(ALPHA_KEY)}")

Data directory created: data/raw
Alpha Vantage API key available: False


## Helper Functions

In [3]:
def safe_stamp():
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")

def safe_filename(prefix: str, meta: Dict[str, str]) -> str:
    mid = "_".join([f"{k}-{str(v).replace(' ', '-')[:30]}" for k,v in meta.items()])
    return f"{prefix}_{mid}_{safe_stamp()}.csv"

def validate_df(df: pd.DataFrame,
                required_cols: list,
                dtypes_map: Dict[str,str] = None,
                min_rows: int = 1) -> dict:
    dtypes_map = dtypes_map or {}
    msgs = {
        "missing_cols": [],
        "bad_dtypes": {},
        "na_count": int(df.isna().sum().sum()),
        "n_rows": int(df.shape[0]),
        "n_cols": int(df.shape[1]),
    }
    msgs["missing_cols"] = [c for c in required_cols if c not in df.columns]
    for col, t in dtypes_map.items():
        if col not in df.columns: 
            msgs["bad_dtypes"][col] = f"missing for dtype {t}"
            continue
        try:
            if t.startswith("datetime"):
                _ = pd.to_datetime(df[col], errors="coerce")
            elif t in ("float", "float64"):
                _ = pd.to_numeric(df[col], errors="coerce")
            elif t in ("int","int64"):
                _ = pd.to_numeric(df[col], errors="coerce").astype("Int64")
            else:
                _ = df[col].astype(t)
            if _.isna().mean() > 0.1:
                msgs["bad_dtypes"][col] = f"too many NA after cast to {t}"
        except Exception as e:
            msgs["bad_dtypes"][col] = f"cast to {t} failed: {e}"
    if msgs["n_rows"] < min_rows:
        msgs["bad_shape"] = f"rows<{min_rows}"
    return msgs

print("Helper functions defined successfully!")

Helper functions defined successfully!


## Part 1: API Data Acquisition

In [4]:
SYMBOL = "AAPL"
use_alpha = bool(ALPHA_KEY)

if use_alpha:
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": SYMBOL,
        "outputsize": "compact",
        "apikey": ALPHA_KEY,
        "datatype": "json"
    }
    try:
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        js = r.json()
        ts_key = [k for k in js.keys() if "Time Series" in k]
        assert ts_key, f"No time series key in response: keys={list(js.keys())[:5]}"
        ts = js[ts_key[0]]
        df_api = (
            pd.DataFrame(ts)
              .T.reset_index()
              .rename(columns={"index":"date", "5. adjusted close":"adj_close"})
              [["date","adj_close"]]
        )
        df_api["date"] = pd.to_datetime(df_api["date"])
        df_api["adj_close"] = pd.to_numeric(df_api["adj_close"], errors="coerce")
        print(f"✅ Successfully fetched data from Alpha Vantage for {SYMBOL}")
    except Exception as e:
        print("Alpha Vantage failed, falling back to yfinance. Error:", e)
        use_alpha = False

if not use_alpha:
    print(f"📊 Fetching {SYMBOL} data from yfinance...")
    stock_data = yf.download(SYMBOL, period="6mo", interval="1d", auto_adjust=True)
    print("Available columns:", list(stock_data.columns))
    
    if isinstance(stock_data.columns, pd.MultiIndex):
        stock_data.columns = [col[0] for col in stock_data.columns]
    
    df_api = stock_data.reset_index()[["Date","Close"]]
    df_api.columns = ["date","adj_close"]
    df_api["date"] = pd.to_datetime(df_api["date"])
    df_api["adj_close"] = pd.to_numeric(df_api["adj_close"], errors="coerce")
    print(f"✅ Successfully fetched data from yfinance for {SYMBOL}")

df_api = df_api.sort_values("date").reset_index(drop=True)
msgs = validate_df(df_api,
                   required_cols=["date","adj_close"],
                   dtypes_map={"date":"datetime64[ns]","adj_close":"float"},
                   min_rows=10)
print("API validation:", msgs)

fname = safe_filename(prefix="api",
                      meta={"source": "alpha" if use_alpha else "yfinance",
                            "ticker": SYMBOL})
out_path = DATA_RAW / fname
df_api.to_csv(out_path, index=False)
print("Saved:", out_path)

print("\nFirst 5 rows of API data:")
print(df_api.head())

[*********************100%***********************]  1 of 1 completed


📊 Fetching AAPL data from yfinance...
Available columns: [('Close', 'AAPL'), ('High', 'AAPL'), ('Low', 'AAPL'), ('Open', 'AAPL'), ('Volume', 'AAPL')]
✅ Successfully fetched data from yfinance for AAPL
API validation: {'missing_cols': [], 'bad_dtypes': {}, 'na_count': 0, 'n_rows': 125, 'n_cols': 2}
Saved: data/raw/api_source-yfinance_ticker-AAPL_20250818-031601.csv

First 5 rows of API data:
        date   adj_close
0 2025-02-18  243.873062
1 2025-02-19  244.272079
2 2025-02-20  245.229736
3 2025-02-21  244.950424
4 2025-02-24  246.496643


## Part 2: Web Scraping

In [5]:
SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "AFE-Course-Notebook/1.0 (contact: your_email@example.com)"}

def parse_first_table(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table")
    assert table is not None, "No <table> element found"
    rows = []
    for tr in table.find_all("tr"):
        cells = [td.get_text(strip=True) for td in tr.find_all(["td","th"])]
        if cells:
            rows.append(cells)
    header, *data = rows
    return pd.DataFrame(data, columns=header)

try:
    print(f"🌐 Attempting to scrape: {SCRAPE_URL}")
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    df_scrape = parse_first_table(resp.text)
    print(f"✅ Successfully scraped table with {df_scrape.shape[0]} rows and {df_scrape.shape[1]} columns")
except Exception as e:
    print("Scrape failed (using inline demo table).", e)
    html = """
    <table>
      <tr><th>Ticker</th><th>Company</th><th>Price</th><th>Market Cap</th></tr>
      <tr><td>AAPL</td><td>Apple Inc.</td><td>150.25</td><td>2.5T</td></tr>
      <tr><td>MSFT</td><td>Microsoft Corp</td><td>280.50</td><td>2.1T</td></tr>
      <tr><td>GOOGL</td><td>Alphabet Inc.</td><td>125.75</td><td>1.6T</td></tr>
      <tr><td>AMZN</td><td>Amazon.com Inc</td><td>95.30</td><td>980B</td></tr>
      <tr><td>TSLA</td><td>Tesla Inc</td><td>220.15</td><td>700B</td></tr>
    </table>
    """
    df_scrape = parse_first_table(html)
    print(f"📋 Using demo table with {df_scrape.shape[0]} rows and {df_scrape.shape[1]} columns")

for col in df_scrape.columns:
    if any(x in col.lower() for x in ["price","close","volume","market","cap","weight","%","chg","change"]):
        cleaned = (
            df_scrape[col]
            .str.replace(r"[^0-9.\-]", "", regex=True)
            .replace({"": None})
        )
        maybe_num = pd.to_numeric(cleaned, errors="coerce")
        if maybe_num.notna().mean() > 0.5:
            df_scrape[col] = maybe_num
            print(f"🔢 Converted column '{col}' to numeric")

msgs2 = validate_df(df_scrape, required_cols=list(df_scrape.columns), dtypes_map={}, min_rows=3)
print("SCRAPE validation:", msgs2)

fname2 = safe_filename(prefix="scrape", meta={"site":"wikipedia","table":"sp500"})
out_path2 = DATA_RAW / fname2
df_scrape.to_csv(out_path2, index=False)
print("Saved:", out_path2)

print("\nFirst 5 rows of scraped data:")
print(df_scrape.head())

🌐 Attempting to scrape: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
✅ Successfully scraped table with 503 rows and 9 columns
SCRAPE validation: {'missing_cols': [], 'bad_dtypes': {}, 'na_count': 0, 'n_rows': 503, 'n_cols': 9}
Saved: data/raw/scrape_site-wikipedia_table-sp500_20250818-031602.csv

First 5 rows of scraped data:
  Symbol                    Security  GICS Sector GICS Sub-Industry  \
0    MMM                          3M  Industrials   Industrial Conglomerates   
1    AOS           A. O. Smith Corp.  Industrials        Building Products   
2    ABT             Abbott Laboratories   Health Care     Health Care Equipment   
3   ABBV                    AbbVie Inc.   Health Care           Biotechnology   
4   ABMD                Abiomed Inc   Health Care     Health Care Equipment   

  Headquarters Location Date added CIK      Founded  
0     Saint Paul, Minnesota  1976-08-09   66740         1902  
1       Milwaukee, Wisconsin  2017-07-26   91142         1916  
2   

## Summary

In [6]:
print("📁 Generated files in data/raw/:")
for file in DATA_RAW.glob("*.csv"):
    print(f"  - {file.name} ({file.stat().st_size} bytes)")

print("\n✅ Data acquisition and ingestion completed successfully!")
print(f"📊 API data: {df_api.shape[0]} rows, {df_api.shape[1]} columns")
print(f"🌐 Scraped data: {df_scrape.shape[0]} rows, {df_scrape.shape[1]} columns")

📁 Generated files in data/raw/:
  - scrape_site-wikipedia_table-sp500_20250818-031602.csv (45504 bytes)
  - api_source-yfinance_ticker-AAPL_20250817-231151.csv (3701 bytes)
  - scrape_site-wikipedia_table-sp500_20250817-231151.csv (179 bytes)
  - api_source-yfinance_ticker-AAPL_20250818-031601.csv (3701 bytes)

✅ Data acquisition and ingestion completed successfully!
📊 API data: 125 rows, 2 columns
🌐 Scraped data: 503 rows, 9 columns
